In [1]:
import json
import random
import re
import spacy
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score

In [2]:
def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')
    cleaned_data = []
    for v in data:
#         print(text)
        text = v["text"]
        entities = v["labels"]
#         print(entities)
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append((valid_start, valid_end, label))
        cleaned_data.append((text, {'entities': valid_entities}))
    return cleaned_data

In [3]:
class Entity:
    count=0
    exact=0
    partial=0
    wrong=0
    extras=0
    
    pcw=0
    tcw=0
    def Precision(self):
#         if self.count != 0:
        if self.count == 0 and self.extras == 0:
            return 1
        else:
            if self.partial!=0:
                partialPercentage=(self.pcw/self.tcw)
            else:
                partialPercentage=1
            p = (self.exact+ self.partial*(partialPercentage))/(self.count-self.partial*(1-partialPercentage)+self.extras)
            return p
#         else:
#             return 1
    def Recall(self):
        if self.count != 0:
            if self.partial!=0:
                partialPercentage=(self.pcw/self.tcw)
            else:
                partialPercentage=1
            r = (self.exact+ self.partial*(partialPercentage))/self.count
            return r
        else:
            return 1
    def Fscore(self):
        p=self.Precision()
        r=self.Recall()
        if p!=0:
            f=2*p*r/(p+r)
        else:
            f=0
        return f
    
# obj = {"PERSON":Entity(),"Number":Entity(),"Email":Entity(),"Location":Entity(),"Skill":Entity(),"Degree":Entity(),"Institute":Entity(),"Designation":Entity(),"ORG":Entity(),"Urls":Entity(),"Certifications":Entity()}
# print(obj.get("PERSON").Fscore())

In [4]:
def evaluate():
    main=[]
    ind=0
    n=0
    temp=0
    t_data = []
    
    for line in open('eval_45.json', 'r',encoding="utf-8"):    # annotated eval file
        t_data.append(json.loads(line))
    examples = trim_entity_spans(t_data)
        
    nlp = spacy.load('model20')                                # model to evaluate
    
    for text, annot in examples:                    # for each resume
        obj = {"name":t_data[ind].get("name"),"PERSON":Entity(),"Number":Entity(),"Email":Entity(),"Location":Entity(),"Skill":Entity(),"Degree":Entity(),"Institute":Entity(),"Designation":Entity(),"ORG":Entity(),"Urls":Entity(),"Certifications":Entity()}
        ind+=1
        actual = sorted((annot.get("entities")),key = lambda st : st[0])
        predicted = []
        doc_to_test=nlp(text)
        print('/n.........................................................../n')
#         print(doc_to_test.ents)
        for ent in doc_to_test.ents:
            predicted.append((ent.start_char, ent.end_char, ent.label_))
#             print(ent.text, ent.start_char, ent.end_char, ent.label_)
#             try:
#                 print(actual[n])
#                 print(predicted[n])
#                 n+=1
#             except:
#                 print("ended")
#             print('\n')
#         break
#     print('\n /////////////////')
    
        i=0
        j=0
        while i<len(actual) and j<len(predicted):

            print("............")
            if actual[i][0]<predicted[j][0]:              #case main 1
                if(actual[i][1]<=predicted[j][0]):
                    print("lets see...",actual[i],predicted[j])
                    if j==0 or predicted[j-1][1]<actual[i][0]:
                        print(actual[i])
                        obj[actual[i][2]].count+=1
                        print("niceeeeeeeeeeeeeeeeeeeeee",obj[actual[i][2]].count)
                    i+=1
                elif(actual[i][1]<predicted[j][1]):
                    print(actual[i])
                    obj[actual[i][2]].count+=1                               #c
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].partial+=1                         #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])   #c
                        obj[actual[i][2]].pcw+=(actual[i][1]-predicted[j][0])*2
                    else:
                        obj[actual[i][2]].wrong+=1                           #c
                    i+=1
                elif(actual[i][1]==predicted[j][1]):
                    print(actual[i])
                    obj[actual[i][2]].count+=1                                   #c
                    if(actual[i][2]==predicted[j][2]):                           #c
                        obj[actual[i][2]].partial+=1                             #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])       #c
                        obj[actual[i][2]].pcw+=(predicted[j][1]-predicted[j][0])*2 #partial cover
                    else:
                        obj[actual[i][2]].wrong+=1                               #c
                    i+=1
                    print(predicted[j])
                    j+=1
                elif(actual[i][1]>predicted[j][1]):
                    print(predicted[j])
                    obj[actual[i][2]].count+=1                                   #c
                    if(actual[i][2]==predicted[j][2]):                           #c
                        obj[actual[i][2]].partial+=1                             #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])       #c
                        obj[actual[i][2]].pcw+=(predicted[j][1]-predicted[j][0])*2 #partial cover
                    else:
                        obj[actual[i][2]].wrong+=1                               #c
                    j+=1
            elif actual[i][0]==predicted[j][0]:           # main case 2
                if(actual[i][1]<predicted[j][1]):
                    print(actual[i])

                    obj[actual[i][2]].count+=1                               #c
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].partial+=1                         #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])   #c
                        obj[actual[i][2]].pcw+=(actual[i][1]-actual[i][0])*2   #cover
                    else:
                        obj[actual[i][2]].wrong+=1

                    i+=1
                elif(actual[i][1]==predicted[j][1]):

                    obj[actual[i][2]].count+=1
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].exact+=1                         #match
                    else:
                        obj[actual[i][2]].wrong+=1

                    print("niceeeeeeeeeeeeeeeeeeeeee",obj[actual[i][2]].count)
                    print(actual[i])
                    i+=1
                    print(predicted[j])
                    j+=1
                elif(actual[i][1]>predicted[j][1]):
                    print(predicted[j])

                    obj[actual[i][2]].count+=1                                 #c
                    if(actual[i][2]==predicted[j][2]):                         #c
                        obj[actual[i][2]].partial+=1                           #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])     #c
                        obj[actual[i][2]].pcw+=(predicted[j][1]-predicted[j][0])*2 
                    else:
                        obj[actual[i][2]].wrong+=1                             #c

                    j+=1
            elif actual[i][0]>predicted[j][0]:           # main case 3
                if(actual[i][0]>=predicted[j][1]):
                    if i==0 or actual[i-1][1]<predicted[j][0]:
                        print("extra : ",predicted[j])
                        print(obj[predicted[j][2]].extras)
                        obj[predicted[j][2]].extras+=1
                    j+=1
                elif(actual[i][1]<predicted[j][1]):
                    print(actual[i])

                    obj[actual[i][2]].count+=1                               #c
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].partial+=1                         #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])   #c
                        obj[actual[i][2]].pcw+=(actual[i][1]-actual[i][0])*2   #cover
                    else:
                        obj[actual[i][2]].wrong+=1                           #c

                    i+=1
                elif(actual[i][1]==predicted[j][1]):
                    print(actual[i])

                    obj[actual[i][2]].count+=1                               #c
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].partial+=1                         #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])   #c
                        obj[actual[i][2]].pcw+=(actual[i][1]-actual[i][0])*2   #cover
                    else:
                        obj[actual[i][2]].wrong+=1                           #c

                    i+=1
                    print(predicted[j])
                    j+=1
                elif(actual[i][1]>predicted[j][1]):
                    print(predicted[j])

                    obj[actual[i][2]].count+=1                               #c
                    if(actual[i][2]==predicted[j][2]):                       #c
                        obj[actual[i][2]].partial+=1                         #c
                        obj[actual[i][2]].tcw+=(actual[i][1]-actual[i][0])+(predicted[j][1]-predicted[j][0])   #c
                        obj[actual[i][2]].pcw+=(predicted[j][1]-actual[i][0])*2
                    else:
                        obj[actual[i][2]].wrong+=1                           #c

                    j+=1

        while(i<len(actual)):
            print("............")
            print(actual[i])
            obj[actual[i][2]].count+=1
            i+=1
        while j<len(predicted):
            print("............")
            print(predicted[j])
            obj[predicted[j][2]].extras+=1
            j+=1
        
        main.append(obj)
    return main

In [5]:
main=evaluate()

/n.........................................................../n
............
niceeeeeeeeeeeeeeeeeeeeee 1
(0, 9, 'PERSON')
(0, 9, 'PERSON')
............
extra :  (52, 80, 'ORG')
0
............
lets see... (81, 99, 'Number') (102, 120, 'Email')
(81, 99, 'Number')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 1
(102, 120, 'Email')
(102, 120, 'Email')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(174, 215, 'Designation')
(174, 215, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(216, 244, 'ORG')
(216, 244, 'ORG')
............
extra :  (337, 353, 'Designation')
0
............
lets see... (1758, 1781, 'Skill') (2089, 2114, 'Designation')
(1758, 1781, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 1
............
lets see... (1783, 1819, 'Skill') (2089, 2114, 'Designation')
(1783, 1819, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 2
............
lets see... (1821, 1856, 'Skill') (2089, 2114, 'Designation')
(1821, 1856, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 3
............
lets see... (1858, 1874, 'Ski

/n.........................................................../n
............
niceeeeeeeeeeeeeeeeeeeeee 1
(14, 56, 'Location')
(14, 56, 'Location')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(63, 75, 'Number')
(63, 75, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(106, 128, 'Email')
(106, 128, 'Email')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(129, 152, 'PERSON')
(129, 152, 'PERSON')
............
(338, 345, 'ORG')
............
(338, 355, 'ORG')
............
(356, 361, 'Designation')
(346, 361, 'Designation')
............
extra :  (2290, 2294, 'Skill')
0
............
extra :  (2299, 2304, 'Skill')
1
............
extra :  (2446, 2482, 'Designation')
0
............
niceeeeeeeeeeeeeeeeeeeeee 3
(2651, 2661, 'ORG')
(2651, 2661, 'ORG')
............
(2662, 2677, 'Designation')
............
lets see... (2662, 2715, 'Designation') (2860, 2884, 'Institute')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(2860, 2884, 'Institute')
(2860, 2884, 'Institute')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(288

/n.........................................................../n
............
extra :  (0, 9, 'PERSON')
0
............
extra :  (46, 59, 'PERSON')
1
............
lets see... (87, 100, 'PERSON') (109, 123, 'Number')
(87, 100, 'PERSON')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 1
(109, 123, 'Number')
(109, 123, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(185, 207, 'Email')
(185, 207, 'Email')
............
(261, 273, 'Location')
............
............
(308, 361, 'Degree')
............
lets see... (308, 376, 'Degree') (379, 399, 'Institute')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(379, 399, 'Institute')
(379, 399, 'Institute')
............
lets see... (846, 887, 'Degree') (890, 910, 'Institute')
(846, 887, 'Degree')
niceeeeeeeeeeeeeeeeeeeeee 2
............
niceeeeeeeeeeeeeeeeeeeeee 2
(890, 910, 'Institute')
(890, 910, 'Institute')
............
lets see... (1296, 1324, 'Certifications') (1370, 1396, 'ORG')
(1296, 1324, 'Certifications')
niceeeeeeeeeeeeee

/n.........................................................../n
............
(0, 16, 'PERSON')
............
lets see... (0, 24, 'PERSON') (33, 46, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(33, 46, 'Number')
(33, 46, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(55, 78, 'Email')
(55, 78, 'Email')
............
lets see... (767, 785, 'ORG') (792, 820, 'Designation')
(767, 785, 'ORG')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 1
(792, 820, 'Designation')
(792, 820, 'Designation')
............
lets see... (1807, 1825, 'ORG') (1832, 1860, 'Designation')
(1807, 1825, 'ORG')
niceeeeeeeeeeeeeeeeeeeeee 2
............
niceeeeeeeeeeeeeeeeeeeeee 2
(1832, 1860, 'Designation')
(1832, 1860, 'Designation')
............
lets see... (2672, 2702, 'ORG') (2710, 2725, 'Designation')
(2672, 2702, 'ORG')
niceeeeeeeeeeeeeeeeeeeeee 3
............
niceeeeeeeeeeeeeeeeeeeeee 3
(2710, 2725, 'Designation')
(2710, 2725, 'Designation')
............
(3244, 3263, 'ORG')
..........

lets see... (32, 40, 'Location') (41, 53, 'Number')
(32, 40, 'Location')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 1
(41, 53, 'Number')
(41, 53, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(54, 79, 'Email')
(54, 79, 'Email')
............
(98, 154, 'Urls')
(135, 154, 'Urls')
............
lets see... (178, 213, 'Urls') (226, 260, 'Institute')
(178, 213, 'Urls')
niceeeeeeeeeeeeeeeeeeeeee 2
............
(226, 252, 'Institute')
............
............
lets see... (378, 390, 'Skill') (466, 542, 'Institute')
(378, 390, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 1
............
lets see... (409, 433, 'Certifications') (466, 542, 'Institute')
(409, 433, 'Certifications')
niceeeeeeeeeeeeeeeeeeeeee 1
............
(466, 490, 'Institute')
............
............
extra :  (603, 619, 'Degree')
0
............
niceeeeeeeeeeeeeeeeeeeeee 3
(631, 651, 'Institute')
(631, 651, 'Institute')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(776, 795, 'ORG')
(776, 795, 'ORG')
............
ni

...

............
niceeeeeeeeeeeeeeeeeeeeee 1
(166, 186, 'Email')
(166, 186, 'Email')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(414, 432, 'ORG')
(414, 432, 'ORG')
............
(443, 467, 'Designation')
............
............
niceeeeeeeeeeeeeeeeeeeeee 2
(3598, 3629, 'ORG')
(3598, 3629, 'ORG')
............
(3630, 3647, 'Designation')
............
lets see... (3630, 3673, 'Designation') (7419, 7453, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 3
(7419, 7453, 'ORG')
(7419, 7453, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 3
(7454, 7471, 'Designation')
(7454, 7471, 'Designation')
............
niceeeeeeeeeeeeeeeeeeeeee 4
(8765, 8790, 'ORG')
(8765, 8790, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 4
(8791, 8811, 'Designation')
(8791, 8811, 'Designation')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(9518, 9536, 'Institute')
(9518, 9536, 'Institute')
............
(9546, 9552, 'Degree')
............
lets see... (9546, 9553, 'Degree') (9566, 9584, 'Institute')
............
niceeeeeeeeeeeeeeeeeeee

/n.........................................................../n
............
niceeeeeeeeeeeeeeeeeeeeee 1
(70, 88, 'PERSON')
(70, 88, 'PERSON')
............
lets see... (95, 122, 'Email') (123, 147, 'Email')
(95, 122, 'Email')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 2
(123, 147, 'Email')
(123, 147, 'Email')
............
lets see... (172, 187, 'Number') (513, 533, 'ORG')
(172, 187, 'Number')
niceeeeeeeeeeeeeeeeeeeeee 1
............
niceeeeeeeeeeeeeeeeeeeeee 1
(513, 533, 'ORG')
(513, 533, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(547, 570, 'Designation')
(547, 570, 'Designation')
............
niceeeeeeeeeeeeeeeeeeeeee 2
(586, 623, 'ORG')
(586, 623, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 2
(665, 688, 'Designation')
(665, 688, 'Designation')
............
lets see... (704, 719, 'ORG') (799, 810, 'ORG')
(704, 719, 'ORG')
niceeeeeeeeeeeeeeeeeeeeee 3
............
lets see... (760, 783, 'Designation') (799, 810, 'ORG')
(760, 783, 'Designation')
niceeeeeee

/n.........................................................../n
............
niceeeeeeeeeeeeeeeeeeeeee 1
(0, 17, 'PERSON')
(0, 17, 'PERSON')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(18, 35, 'Email')
(18, 35, 'Email')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(47, 61, 'Number')
(47, 61, 'Number')
............
extra :  (112, 140, 'Designation')
0
............
extra :  (1289, 1313, 'ORG')
0
............
niceeeeeeeeeeeeeeeeeeeeee 1
(1340, 1349, 'ORG')
(1340, 1349, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(1353, 1363, 'Designation')
(1353, 1363, 'Designation')
............
extra :  (1395, 1417, 'ORG')
1
............
niceeeeeeeeeeeeeeeeeeeeee 2
(1444, 1452, 'ORG')
(1444, 1452, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 2
(1456, 1481, 'Designation')
(1456, 1481, 'Designation')
............
niceeeeeeeeeeeeeeeeeeeeee 3
(1565, 1574, 'ORG')
(1565, 1574, 'ORG')
............
niceeeeeeeeeeeeeeeeeeeeee 3
(1578, 1595, 'Designation')
(1578, 1595, 'Designation')
............
niceeeeeeeeeeeeeee

/n.........................................................../n
............
niceeeeeeeeeeeeeeeeeeeeee 1
(0, 12, 'PERSON')
(0, 12, 'PERSON')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(13, 29, 'Location')
(13, 29, 'Location')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(32, 48, 'Number')
(32, 48, 'Number')
............
niceeeeeeeeeeeeeeeeeeeeee 1
(51, 74, 'Email')
(51, 74, 'Email')
............
lets see... (129, 148, 'Designation') (218, 246, 'Designation')
(129, 148, 'Designation')
niceeeeeeeeeeeeeeeeeeeeee 1
............
extra :  (218, 246, 'Designation')
0
............
lets see... (1090, 1096, 'Skill') (1453, 1476, 'Certifications')
(1090, 1096, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 1
............
lets see... (1101, 1106, 'Skill') (1453, 1476, 'Certifications')
(1101, 1106, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 2
............
niceeeeeeeeeeeeeeeeeeeeee 1
(1453, 1476, 'Certifications')
(1453, 1476, 'Certifications')
............
niceeeeeeeeeeeeeeeeeeeeee 2
(1479, 1495, 'Certifications')
(1479, 1

niceeeeeeeeeeeeeeeeeeeeee 2
............
lets see... (211, 238, 'Skill') (1387, 1388, 'Skill')
(211, 238, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 1
............
lets see... (243, 254, 'Skill') (1387, 1388, 'Skill')
(243, 254, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 2
............
lets see... (351, 367, 'Skill') (1387, 1388, 'Skill')
(351, 367, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 3
............
lets see... (453, 459, 'Skill') (1387, 1388, 'Skill')
(453, 459, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 4
............
lets see... (464, 465, 'Skill') (1387, 1388, 'Skill')
(464, 465, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 5
............
lets see... (466, 482, 'Skill') (1387, 1388, 'Skill')
(466, 482, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 6
............
lets see... (533, 552, 'Skill') (1387, 1388, 'Skill')
(533, 552, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 7
............
lets see... (633, 639, 'Skill') (1387, 1388, 'Skill')
(633, 639, 'Skill')
niceeeeeeeeeeeeeeeeeeeeee 8
............
lets see... (709, 710, 'Skill') (1387, 

In [6]:
def func(obt,ent):
    obj=obt.get(ent)
    if obj.count==0:   # entity not present
        return 0
    elif obj.exact >= (obj.count)*(4/5):   # atleast 80% is exact
        if obj.exact+obj.partial >= (obj.count+obj.extras)*(4/5):    # atleast 80% of parsed is tending to correct
            return 4
        else:
            return 3
    elif (obj.partial+obj.exact+obj.wrong) == 0:   # entity not parsed
        return 1
    elif (obj.exact+obj.partial) >= (obj.count)*(3/10):   # atleast 30% of actual is exact or partially correct
        return 3
    else:
        return 2

In [7]:
def totalScore(main):
    n=0
    score=0
    Fent={'PERSON':0,"Number":0,"Email":0,"Location":0,"Skill":0,"Degree":0,"Institute":0,"Designation":0,"ORG":0,"Urls":0,"Certifications":0}
    li=[]
    for obj in main:
        Fobj=0
        for key in obj:
            if key!="name":
                temp=obj.get(key).Fscore()
                Fent[key]+=temp
                Fobj+=temp
        score+=Fobj
        li.append(Fobj/11)
        n+=1
    return(li,Fent,score)

In [8]:
import xlwt 
  
workbook = xlwt.Workbook()  
  
sheet = workbook.add_sheet("Sheet Name") 
  
sheet.col(1).width = 256 * 20
sheet.col(2).width = 256 * 12
sheet.col(3).width = 256 * 12
sheet.col(4).width = 256 * 12
sheet.col(5).width = 256 * 12
sheet.col(6).width = 256 * 12
sheet.col(7).width = 256 * 12
sheet.col(8).width = 256 * 12
sheet.col(9).width = 256 * 12
sheet.col(10).width = 256 * 12
sheet.col(11).width = 256 * 12
sheet.col(12).width = 256 * 12
sheet.row(0).height_mismatch = True
sheet.row(46).height_mismatch = True
sheet.row(0).height = 256*2
sheet.row(46).height = 256*2

al = xlwt.Alignment () 
al.horz = 0x02       # arranged horizontally centered 
al.vert = 0x01       # set vertical centering 

style = xlwt.XFStyle()
pattern = xlwt.Pattern()
pattern.pattern = xlwt.Pattern.SOLID_PATTERN
pattern.pattern_fore_colour = xlwt.Style.colour_map['light_turquoise']
style.pattern = pattern
style.alignment = al

sheet.write(0, 0, 'S.No.', style) 

style2 = xlwt.XFStyle()
pattern2 = xlwt.Pattern()
pattern2.pattern = xlwt.Pattern.SOLID_PATTERN
pattern2.pattern_fore_colour = xlwt.Style.colour_map['ice_blue']
style2.pattern = pattern2
style2.alignment = al

sheet.write(0, 1, 'Name', style2) 

style = xlwt.XFStyle()
pattern = xlwt.Pattern()
pattern.pattern = xlwt.Pattern.SOLID_PATTERN
pattern.pattern_fore_colour = xlwt.Style.colour_map['light_orange']
style.pattern = pattern
style.alignment = al

i=2
j=0
for obj in main:
    for key in obj.keys():
        if key!="name":
            sheet.write(j,i,key,style)
            i+=1
    break
    
style = xlwt.XFStyle()
style.alignment = al

i=1
j=0
for obj in main:
    sheet.write(i,j,i,xlwt.easyxf("pattern: pattern solid, fore_colour gray25;align: horiz center"))
    sheet.write(i,j+1,obj.get('name'),xlwt.easyxf("align: horiz left"))
    
#     sheet.write(i,j+2,func(obj,'PERSON'),style)
#     sheet.write(i,j+3,func(obj,'Number'),style)
#     sheet.write(i,j+4,func(obj,'Email'),style)
#     sheet.write(i,j+5,func(obj,'Location'),style)
#     sheet.write(i,j+6,func(obj,'Skill'),style)
#     sheet.write(i,j+7,func(obj,'Degree'),style)
#     sheet.write(i,j+8,func(obj,'Institute'),style)
#     sheet.write(i,j+9,func(obj,'Designation'),style)
#     sheet.write(i,j+10,func(obj,'ORG'),style)
#     sheet.write(i,j+11,func(obj,'Urls'),style)
#     sheet.write(i,j+12,func(obj,'Certifications'),style)

    sheet.write(i,j+2,obj.get('PERSON').Fscore(),style)
    sheet.write(i,j+3,obj.get('Number').Fscore(),style)
    sheet.write(i,j+4,obj.get('Email').Fscore(),style)
    sheet.write(i,j+5,obj.get('Location').Fscore(),style)
    sheet.write(i,j+6,obj.get('Skill').Fscore(),style)
    sheet.write(i,j+7,obj.get('Degree').Fscore(),style)
    sheet.write(i,j+8,obj.get('Institute').Fscore(),style)
    sheet.write(i,j+9,obj.get('Designation').Fscore(),style)
    sheet.write(i,j+10,obj.get('ORG').Fscore(),style)
    sheet.write(i,j+11,obj.get('Urls').Fscore(),style)
    sheet.write(i,j+12,obj.get('Certifications').Fscore(),style)

    i+=1
    j=0
    
(li,Fent,score)=totalScore(main)

style2=xlwt.easyxf("pattern: pattern solid, fore_colour pale_blue;align: horiz center")

sheet.write(i,j+2,round(Fent.get('PERSON')*100/len(li),3),style2)
sheet.write(i,j+3,round(Fent.get('Number')*100/len(li),3),style2)
sheet.write(i,j+4,round(Fent.get('Email')*100/len(li),3),style2)
sheet.write(i,j+5,round(Fent.get('Location')*100/len(li),3),style2)
sheet.write(i,j+6,round(Fent.get('Skill')*100/len(li),3),style2)
sheet.write(i,j+7,round(Fent.get('Degree')*100/len(li),3),style2)
sheet.write(i,j+8,round(Fent.get('Institute')*100/len(li),3),style2)
sheet.write(i,j+9,round(Fent.get('Designation')*100/len(li),3),style2)
sheet.write(i,j+10,round(Fent.get('ORG')*100/len(li),3),style2)
sheet.write(i,j+11,round(Fent.get('Urls')*100/len(li),3),style2)
sheet.write(i,j+12,round(Fent.get('Certifications')*100/len(li),3),style2)
sheet.write(i,j+13,str(round(score*100/(11*len(li)),2))+" %",xlwt.easyxf("pattern: pattern solid, fore_colour lavender;align: horiz center"))
i=1
for f in li:
    sheet.write(i,j+13,round(f*100,3),style2)
    i+=1

workbook.save("jaimatadi.xls") 

In [ ]:
# ~The End :)

In [136]:
from spacy import displacy
t_data=[]
for line in open('eval_45.json', 'r',encoding="utf-8"):
        t_data.append(json.loads(line))
examples = trim_entity_spans(t_data)
nlp = spacy.load('model20')
# print(t_data)
temp=0
for text, annot in examples:
    doc_to_test=nlp(text)
#     print(annot.get("entities"))
    if temp==0:
        displacy.serve(doc_to_test, style="ent")
        break
    else:
        temp+=1
#     break
# print(examples[0][1].get("entities"))


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [138]:
obj=main[0]
print(obj['name'])
print(obj['Institute'].count)
print(obj['Institute'].partial)
print(obj['Institute'].wrong)
print(obj['Institute'].exact)
print(obj['Institute'].extras)
print(obj['Institute'].pcw) 
print(obj['Institute'].tcw)
print(obj['Institute'].pcw) 
print(obj['Institute'].Precision())
print(obj['Institute'].Recall())

Aimee Siy - Updated CV May 2019 -3
1
0
0
1
1
0
0
0
0.5
1.0
